In [41]:
from dotenv import load_dotenv
import os
from espn_api.football import League

In [42]:
load_dotenv()

league_id = os.getenv('LEAGUE_ID')
swid = os.getenv('SWID')
espn_s2 = os.getenv('ESPN_S2')
year = 2024

In [43]:
league = League(league_id, year, espn_s2, swid)

In [44]:
import pandas as pd

adp_df = pd.read_csv("../../FantasyPros_2024_Overall_ADP_Rankings.csv")
adp_df_2023 = pd.read_csv("../../FantasyPros_2023_Overall_ADP_Rankings.csv")


In [45]:
adp_df.head()

,Rank,Player,Team,Bye,POS,Yahoo,Sleeper,RTSports,AVG
0,1.0,Christian McCaffrey,SF,9,RB1,1.0,1.0,1.0,1.0
1,2.0,Tyreek Hill,MIA,6,WR1,2.0,2.0,3.0,2.3
2,3.0,CeeDee Lamb,DAL,7,WR2,3.0,3.0,2.0,2.7
3,4.0,Breece Hall,NYJ,12,RB2,4.0,8.0,4.0,5.3
4,5.0,Bijan Robinson,ATL,12,RB3,5.0,6.0,5.0,5.3


In [46]:
adp_df_2023.head()

,Rank,Player,Team,Bye,POS,Yahoo,Sleeper,RTSports,AVG
0,1.0,Justin Jefferson,MIN,13,WR1,1.0,1.0,1.0,1.0
1,2.0,Christian McCaffrey,SF,9,RB1,2.0,2.0,2.0,2.0
2,3.0,Ja'Marr Chase,CIN,7,WR2,3.0,3.0,3.0,3.0
3,4.0,Austin Ekeler,WAS,14,RB2,4.0,4.0,4.0,4.0
4,5.0,Tyreek Hill,MIA,10,WR3,5.0,6.0,5.0,5.3


In [47]:
adp_df['Position'] = adp_df['POS'].str.replace(r'\d+', '', regex=True)
adp_df['Position'] = adp_df['Position'].str.strip()

adp_df_2023['Position'] = adp_df_2023['POS'].str.replace(r'\d+', '', regex=True)
adp_df_2023['Position'] = adp_df_2023['Position'].str.strip()

In [48]:
adp_df.head(5)

,Rank,Player,Team,Bye,POS,Yahoo,Sleeper,RTSports,AVG,Position
0,1.0,Christian McCaffrey,SF,9,RB1,1.0,1.0,1.0,1.0,RB
1,2.0,Tyreek Hill,MIA,6,WR1,2.0,2.0,3.0,2.3,WR
2,3.0,CeeDee Lamb,DAL,7,WR2,3.0,3.0,2.0,2.7,WR
3,4.0,Breece Hall,NYJ,12,RB2,4.0,8.0,4.0,5.3,RB
4,5.0,Bijan Robinson,ATL,12,RB3,5.0,6.0,5.0,5.3,RB


In [49]:
espn_team_id_to_name = {}

for team in league.teams:
    espn_team_id_to_name[team.team_id] = team.team_name

espn_team_id_to_name

{1: 'TaylorMade 3',
 2: 'Rigged AF',
 3: 'Injury Prone',
 4: 'jack goff',
 5: 'Chubby Diggs',
 6: "Don't Puka The Bear",
 7: 'Saquon Deez Nuts',
 8: 'No Punt Intended',
 9: 'Washed',
 10: 'League Median'}

In [50]:
espn_player_id_to_name = {}

all_free_agents = league.free_agents(size=2000)

all_rostered_players = []

for team in league.teams:
    
    for player in team.roster:
        all_rostered_players.append(player)

all_players = all_free_agents + all_rostered_players

for player in all_players:
    espn_player_id_to_name[player.playerId] = player.name
    
espn_name_to_player_id = {v: k for k, v in espn_player_id_to_name.items()}

In [51]:
def get_all_team_picks_so_far(draft, team_id, current_pick_num, team_count):
    
    team_picks = []
    
    for pick in draft[:current_pick_num]:
        if pick.team.team_id == team_id:
            team_picks.append(pick)
    
    if not team_picks:
        # Find the team name for the given team_id
        team_name = None
        for pick in draft:
            if pick.team.team_id == team_id:
                team_name = pick.team.team_name
                break
        # If not found in draft, try to get from espn_team_id_to_name if available
        if team_name is None:
            team_name = espn_team_id_to_name.get(team_id, f"Team {team_id}")
        return f"{team_name} has made no picks so far"
    
    team_picks.sort(key=lambda x: (x.round_num, x.round_pick))
    
    blurb = ""
    
    for pick in team_picks:
        
        try:
            
            overall_pick_num = team_count * (pick.round_num - 1) + pick.round_pick
            player_name = espn_player_id_to_name[pick.playerId]
                    
            player_row = adp_df[adp_df['Player'] == player_name]
            
            player_average_adp = player_row['AVG'].values[0]
            player_position = player_row['Position'].values[0]
            player_position_rank = player_row['POS'].values[0]
            
            blurb += f"With pick {overall_pick_num}, {pick.team.team_name} drafted {player_position} {espn_player_id_to_name[pick.playerId]} (ADP: {player_average_adp}, {player_position_rank}). "
        
        except:
            print("Could not add player with name: " + player_name)
        
    return blurb

In [52]:
def get_recent_draft_context(draft, current_pick_num, last_n_picks):
    # Get the last 3 picks before the current pick
    recent_picks = list(reversed(draft[max(0, current_pick_num-last_n_picks):current_pick_num]))
    picks_info = []
    for pick in recent_picks:
        player_name = espn_player_id_to_name.get(pick.playerId, pick.playerName if hasattr(pick, 'playerName') else "Unknown")
        # Find player info in adp_df
        player_row = adp_df[adp_df['Player'] == player_name]
        if not player_row.empty:
            adp = player_row['AVG'].values[0]
            position = player_row['Position'].values[0]
        else:
            adp = "N/A"
            position = "N/A"
        picks_info.append(f"{player_name} ({position}, ADP: {adp})")
    return "Most recent picks in the draft (most recent first): " + ", ".join(picks_info) if picks_info else "No picks have been made yet."
    

In [53]:
recent_draft_context_blurb = get_recent_draft_context(league.draft, 5, 5)
recent_draft_context_blurb

'Most recent picks in the draft (most recent first): Bijan Robinson (RB, ADP: 5.3), Breece Hall (RB, ADP: 5.3), CeeDee Lamb (WR, ADP: 2.7), Tyreek Hill (WR, ADP: 2.3), Christian McCaffrey (RB, ADP: 1.0)'

In [54]:
def get_next_available_players(draft, current_pick_num):
    players_picked = []
    for pick in draft[:current_pick_num]:
        players_picked.append(pick.playerName)
        
    positions = ['QB', 'RB', 'WR', 'TE', 'K']
    position_blurbs = []

    for pos in positions:
        next_best = adp_df[
            (adp_df['Position'] == pos) & (~adp_df['Player'].isin(players_picked))
        ].sort_values(by='AVG').head(3)
        players_info = []
        for _, row in next_best.iterrows():
            player_name = row['Player']
            adp = row['AVG']
            pos_rank = row['POS']
            players_info.append(f"{player_name} (ADP: {adp}, {pos_rank})")
        if players_info:
            position_blurbs.append(f"{pos}: " + ", ".join(players_info))

    blurb = "The next best available players are:\n" + "\n".join(position_blurbs)

    return blurb
    
    
    
    
    
    
    
    
    
    
    

In [55]:
available_players_blurb = get_next_available_players(league.draft, 5)

In [56]:
get_all_team_picks_so_far(league.draft, 1, 30, 10)

'With pick 9, TaylorMade 3 drafted RB Jonathan Taylor (ADP: 10.0, RB4). With pick 12, TaylorMade 3 drafted WR A.J. Brown (ADP: 10.0, WR6). With pick 29, TaylorMade 3 drafted RB James Cook (ADP: 36.0, RB14). '

In [57]:
# Summarize 2023 Draft

league_2023 = League(league_id, 2023, espn_s2, swid)

draft_blurb_2023 = "The 2023 draft was a 9-team, half PPR league with 1 QB, 2 RB, 2 WR, 1 TE, 1 K, 1 D/ST, 1 Flex (RB/WR/TE), and 7 bench spots."

for pick in league_2023.draft:
    
    if pick.playerId in espn_player_id_to_name and pick.team.team_name != "League Median" and "D/ST" not in pick.playerName:
        
        playerId = pick.playerId
        playerName = espn_player_id_to_name[playerId]
        
        team_id = pick.team.team_id
        
        print(playerName)
        
        player_row = adp_df[adp_df['Player'] == playerName]
        
        if not player_row.empty:
            player_position = player_row['Position'].values[0]
        
            draft_blurb_2023 += f"{player_position} {pick.playerName} was drafted by team with id '{team_id}' in round {pick.round_num} pick {pick.round_pick}.\n"
        else:
            draft_blurb_2023 += f"{pick.playerName} was drafted by team with id '{team_id}' in round {pick.round_num} pick {pick.round_pick}.\n"
        
    
    
    
    

Christian McCaffrey
Justin Jefferson
Nick Chubb
Austin Ekeler
Derrick Henry
Tyreek Hill
Bijan Robinson
Saquon Barkley
Josh Jacobs
Ja'Marr Chase
Travis Kelce
Tony Pollard
Jonathan Taylor
Cooper Kupp
Davante Adams
Stefon Diggs
A.J. Brown
CeeDee Lamb
Deebo Samuel Sr.
Garrett Wilson
Joe Mixon
Amon-Ra St. Brown
Jaylen Waddle
DK Metcalf
Tee Higgins
Jalen Hurts
Jahmyr Gibbs
Patrick Mahomes
Chris Olave
Travis Etienne Jr.
Mark Andrews
Josh Allen
Aaron Jones
Najee Harris
Calvin Ridley
DeVonta Smith
Kenneth Walker III
Rhamondre Stevenson
Amari Cooper
Lamar Jackson
George Kittle
Joe Burrow
Jerry Jeudy
Dameon Pierce
Miles Sanders
James Conner
Keenan Allen
Breece Hall
Alexander Mattison
Alvin Kamara
DJ Moore
Terry McLaurin
DeAndre Hopkins
Cam Akers
Darren Waller
Justin Fields
Justin Herbert
J.K. Dobbins
Isiah Pacheco
Rachaad White
T.J. Hockenson
Mike Williams
Mike Evans
Javonte Williams
Dalvin Cook
Brandon Aiyuk
James Cook
D'Andre Swift
Kyle Pitts
Christian Watson
Dallas Goedert
David Montgomery
Dra

In [58]:
# fantasy_draft_predictor.py

import os
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser

# Load environment variables
load_dotenv()

# Retrieve API key
openai_key = os.getenv("OPENAI_API_KEY")
if not openai_key:
    raise ValueError("Missing OPENAI_API_KEY in environment variables.")

# Define structured output model
class PredictionResponse(BaseModel):
    player_name: str = Field(description="The name of the player predicted to be drafted next")
    position: str = Field(description="The position of that player (e.g., QB, RB, WR, TE, K, DST)")
    reasoning: str = Field(description="Reasoning for why this player will be picked")
    percent_chance: float = Field(description="Percent likelihood (0-100) that this player will be picked next")

class PredictionResponseList(BaseModel):
    predictions: list[PredictionResponse] = Field(description="List of predictions for the next player(s) to be drafted")

# Create the parser
parser = PydanticOutputParser(pydantic_object=PredictionResponseList)

# Prompt template: include instructions
query_prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        "You are a fantasy football expert who predicts the next draft pick "
        "given the draft context, available players, and historical trends. "
        "Always output valid JSON matching the given schema."
    ),
    (
        "human",
        "Draft context:\n{query}\n\n"
        "Format your answer exactly as JSON in this structure:\n{format_instructions}"
    )
]).partial(format_instructions=parser.get_format_instructions())

# Initialize the language model
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    api_key=openai_key,
    max_retries=3,
    timeout=60
)

# Create the chain with parser
predict_chain = query_prompt | llm | parser

# Example usage
if __name__ == "__main__":
    example_query = """
    Next pick: Owner Mike (Pick #24, Round 3, 12-team PPR).
    Roster so far: QB: None | RB: Jonathan Taylor | WR: Justin Jefferson | TE: None | FLEX: None | Bench: None.
    Starters needed: QB(1), RB(1), WR(1), TE(1), FLEX(1), DST(1), K(1).
    Last 3 picks: Bijan Robinson (RB), Derrick Henry (RB), Nick Chubb (RB) — RB run in progress.
    Available top players (ADP):
      QB: Josh Allen (20), Jalen Hurts (22)
      RB: Breece Hall (18), Josh Jacobs (21)
      WR: A.J. Brown (15), Amon-Ra St. Brown (17)
      TE: Mark Andrews (25), T.J. Hockenson (35)
    News: Mike tends to draft RBs early.
    """

    prediction: PredictionResponseList = predict_chain.invoke({"query": example_query})
    for prediction in prediction.predictions:
        print(prediction)


player_name='Breece Hall' position='RB' reasoning='Mike tends to draft RBs early, and with the current RB run in progress, Breece Hall is a top available option that fits his strategy.' percent_chance=70.0
player_name='Josh Jacobs' position='RB' reasoning='Another strong RB option available, but with the current trend of drafting RBs, Hall is slightly more likely to be picked first.' percent_chance=25.0
player_name='Josh Allen' position='QB' reasoning='While Mike needs a QB, the strong RB run may lead him to prioritize another RB over a QB at this pick.' percent_chance=5.0


In [59]:
# Define structured output model
class DraftSummary(BaseModel):
    key_takeaways: list[str] = Field(
        description="List of the most important trends, strategies, and surprises from the draft."
    )

# Create parser
parser = PydanticOutputParser(pydantic_object=DraftSummary)

# Create prompt template
summary_prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        "You are a fantasy football analyst. You will be given a recap of a previous fantasy draft. "
        "Your job is to summarize it into the most important takeaways, trends, and surprises "
        "that would help prepare for future drafts. The goal is to identify patterns of drafting strategy "
        "for this league."
    ),
    (
        "human",
        "Draft Recap:\n{draft_blurb}\n\n"
        "Summarize the above into concise key takeaways.\n\n"
        "Output exactly in this JSON format:\n{format_instructions}"
    )
]).partial(format_instructions=parser.get_format_instructions())

# Build chain
summary_chain = summary_prompt | llm | parser


summary: DraftSummary = summary_chain.invoke({"draft_blurb": draft_blurb_2023})

summary_2023 = ""
for takeaway in summary.key_takeaways:
    summary_2023 += f"{takeaway} "

print(summary_2023)

Running backs dominated the early rounds, with 7 RBs taken in the first 8 picks. Wide receivers were also prioritized, with 6 WRs selected in the first 4 rounds, indicating a strong emphasis on the position. Teams showed a willingness to draft QBs early, with 4 QBs taken in the first 7 rounds, suggesting a trend towards securing top-tier quarterbacks. Tight ends were drafted in the early rounds, with Travis Kelce and Mark Andrews going in rounds 2 and 4 respectively, highlighting their value in this format. There was a notable run on WRs in rounds 5 and 6, indicating a depth of talent at the position that teams capitalized on. Several teams waited until the later rounds to draft kickers and defenses, reflecting a strategy of prioritizing skill positions first. 


In [60]:
league_format_blurb = "This is a 9-man, half PPR league with 1 QB, 2 RB, 2 WR, 1 TE, 1 K, 1 D/ST, 1 Flex (RB/WR/TE), and 7 bench spots."

actual_draft_picks = []
for pick in league.draft:
    if pick.playerId in espn_player_id_to_name and pick.team.team_name != "League Median" and "D/ST" not in pick.playerName:
        actual_draft_picks.append(pick)

correct_predictions = 0
correct_top_3_predictions = 0
total_predictions = 0

for i, pick in enumerate(actual_draft_picks):
    
    blurb = league_format_blurb + "\n"
    blurb += "The previous draft had the following key takeaways: " + summary_2023 + "\n"
    team = pick.team
    team_name = team.team_name
    team_id = team.team_id
    playerId = pick.playerId
            
    draft_situation_blurb = f"It is now team {team_name}'s turn to draft. It is currently pick {i+1} overall."
    team_situation_blurb = get_all_team_picks_so_far(actual_draft_picks, team_id=team_id, current_pick_num=i, team_count=10)
    
    blurb += draft_situation_blurb + "\n" + team_situation_blurb + "\n"
    
    recent_draft_context_blurb = get_recent_draft_context(actual_draft_picks, i, 3)
    blurb += recent_draft_context_blurb + "\n"

    players_available_blurb = get_next_available_players(actual_draft_picks, i)
    
    blurb += players_available_blurb + "\n"
    
    blurb += f"Who is {team_name} going to pick next? Give the 3 most likely options with percentages."
    
    prediction = predict_chain.invoke({"query": blurb})
    
    prediction_names = [prediction.player_name for prediction in prediction.predictions]
    max_percent_chance = 0
    for prediction in prediction.predictions:
        if prediction.percent_chance > max_percent_chance:
            max_percent_chance = prediction.percent_chance
            highest_prediction_name = prediction.player_name
            highest_prediction_position = prediction.position
            highest_prediction_reasoning = prediction.reasoning
    
    print("Blurb: " + blurb)
    print("Actual: " + pick.playerName)
    print("Predicted: " + str(prediction_names))
    
    if highest_prediction_name == pick.playerName:
        correct_predictions += 1
        print(f"Correct prediction!")
    if pick.playerName in prediction_names:
        correct_top_3_predictions += 1
        print(f"Correct top 3 prediction!")
    else:
        print(f"Incorrect prediction!")
        
    total_predictions += 1
    print("Correct prediction count: " + str(correct_predictions) + ", Correct top 3 prediction count: " + str(correct_top_3_predictions) + ", Total predictions: " + str(total_predictions))
    print("--------------------------------")

print(f"Correct predictions: {correct_predictions}")
print(f"Correct top 3 predictions: {correct_top_3_predictions}")
print(f"Total predictions: {total_predictions}")
print(f"Accuracy: {correct_predictions/total_predictions}")
print(f"Top 3 accuracy: {correct_top_3_predictions/total_predictions}")

Blurb: This is a 9-man, half PPR league with 1 QB, 2 RB, 2 WR, 1 TE, 1 K, 1 D/ST, 1 Flex (RB/WR/TE), and 7 bench spots.
The previous draft had the following key takeaways: Running backs dominated the early rounds, with 7 RBs taken in the first 8 picks. Wide receivers were also prioritized, with 6 WRs selected in the first 4 rounds, indicating a strong emphasis on the position. Teams showed a willingness to draft QBs early, with 4 QBs taken in the first 7 rounds, suggesting a trend towards securing top-tier quarterbacks. Tight ends were drafted in the early rounds, with Travis Kelce and Mark Andrews going in rounds 2 and 4 respectively, highlighting their value in this format. There was a notable run on WRs in rounds 5 and 6, indicating a depth of talent at the position that teams capitalized on. Several teams waited until the later rounds to draft kickers and defenses, reflecting a strategy of prioritizing skill positions first. 
It is now team Washed's turn to draft. It is currently p